In [258]:
import streamlit as st
import pandas as pd
import shap
import matplotlib as plt
from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor
import joblib


In [259]:
# Load the house price Dataset
boston_test = pd.read_csv('boston-housing/test.csv')
# X = pd.DataFrame(boston.data, columns = boston.feature_names)
# Y = pd.DataFrame(boston.target, columns=["MEDV"])

boston_test.columns = boston_test.columns.str.upper()
boston_test.columns = boston_test.columns.str.strip()
boston_test.astype('float64')
boston_test = boston_test.drop('ID', axis=1)
boston_test

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT
0,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
1,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21
2,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,396.90,19.15
3,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93
4,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,0.17899,0.0,9.69,0,0.585,5.670,28.8,2.7986,6,391,19.2,393.29,17.60
169,0.28960,0.0,9.69,0,0.585,5.390,72.9,2.7986,6,391,19.2,396.90,21.14
170,0.23912,0.0,9.69,0,0.585,6.019,65.3,2.4091,6,391,19.2,396.90,12.92
171,0.22438,0.0,9.69,0,0.585,6.027,79.7,2.4982,6,391,19.2,396.90,14.33


In [260]:
# Load the house price Dataset
boston_train = pd.read_csv('boston-housing/train.csv')
# X = pd.DataFrame(boston.data, columns = boston.feature_names)
# Y = pd.DataFrame(boston.target, columns=["MEDV"])
boston_train.columns = boston_train.columns.str.upper()
boston_train.columns = boston_train.columns.str.strip()
boston_train.astype('float64')
boston_train = boston_train.drop('ID', axis=1)
boston_train

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,0.17783,0.0,9.69,0,0.585,5.569,73.5,2.3999,6,391,19.2,395.77,15.10,17.5
329,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
330,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
331,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9


In [261]:
boston_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     173 non-null    float64
 1   ZN       173 non-null    float64
 2   INDUS    173 non-null    float64
 3   CHAS     173 non-null    int64  
 4   NOX      173 non-null    float64
 5   RM       173 non-null    float64
 6   AGE      173 non-null    float64
 7   DIS      173 non-null    float64
 8   RAD      173 non-null    int64  
 9   TAX      173 non-null    int64  
 10  PTRATIO  173 non-null    float64
 11  BLACK    173 non-null    float64
 12  LSTAT    173 non-null    float64
dtypes: float64(10), int64(3)
memory usage: 17.7 KB


In [262]:
boston_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     333 non-null    float64
 1   ZN       333 non-null    float64
 2   INDUS    333 non-null    float64
 3   CHAS     333 non-null    int64  
 4   NOX      333 non-null    float64
 5   RM       333 non-null    float64
 6   AGE      333 non-null    float64
 7   DIS      333 non-null    float64
 8   RAD      333 non-null    int64  
 9   TAX      333 non-null    int64  
 10  PTRATIO  333 non-null    float64
 11  BLACK    333 non-null    float64
 12  LSTAT    333 non-null    float64
 13  MEDV     333 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 36.5 KB


In [263]:
boston = pd.concat([boston_test, boston_train])
boston.info()

<class 'pandas.core.frame.DataFrame'>
Index: 506 entries, 0 to 332
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  BLACK    506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     333 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 59.3 KB


In [264]:
boston.columns = boston.columns.str.upper()
boston.columns = boston.columns.str.strip()
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV
0,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,NaN
1,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,NaN
2,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,396.90,19.15,NaN
3,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93,NaN
4,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10,NaN


In [265]:
boston.info()

<class 'pandas.core.frame.DataFrame'>
Index: 506 entries, 0 to 332
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  BLACK    506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     333 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 59.3 KB


In [266]:
boston = boston.astype('float64')

In [267]:
boston.info()

<class 'pandas.core.frame.DataFrame'>
Index: 506 entries, 0 to 332
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  BLACK    506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     333 non-null    float64
dtypes: float64(14)
memory usage: 59.3 KB


In [268]:
boston = boston.dropna()

In [269]:
boston.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333 entries, 0 to 332
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     333 non-null    float64
 1   ZN       333 non-null    float64
 2   INDUS    333 non-null    float64
 3   CHAS     333 non-null    float64
 4   NOX      333 non-null    float64
 5   RM       333 non-null    float64
 6   AGE      333 non-null    float64
 7   DIS      333 non-null    float64
 8   RAD      333 non-null    float64
 9   TAX      333 non-null    float64
 10  PTRATIO  333 non-null    float64
 11  BLACK    333 non-null    float64
 12  LSTAT    333 non-null    float64
 13  MEDV     333 non-null    float64
dtypes: float64(14)
memory usage: 39.0 KB


In [270]:
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
3,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
4,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9


In [271]:
boston.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333 entries, 0 to 332
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     333 non-null    float64
 1   ZN       333 non-null    float64
 2   INDUS    333 non-null    float64
 3   CHAS     333 non-null    float64
 4   NOX      333 non-null    float64
 5   RM       333 non-null    float64
 6   AGE      333 non-null    float64
 7   DIS      333 non-null    float64
 8   RAD      333 non-null    float64
 9   TAX      333 non-null    float64
 10  PTRATIO  333 non-null    float64
 11  BLACK    333 non-null    float64
 12  LSTAT    333 non-null    float64
 13  MEDV     333 non-null    float64
dtypes: float64(14)
memory usage: 39.0 KB


In [272]:
boston.to_csv('boston_full.csv', index=False)

In [273]:
X = boston.columns[:-1].tolist()
X

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'BLACK',
 'LSTAT']

In [274]:
X = pd.DataFrame(boston, columns=X)
X

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
3,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
4,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,0.17783,0.0,9.69,0.0,0.585,5.569,73.5,2.3999,6.0,391.0,19.2,395.77,15.10
329,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
330,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
331,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64


In [275]:
# specify target column
Y = pd.DataFrame(boston, columns=['MEDV'])

In [276]:
Y

,MEDV
0,24.0
1,21.6
2,33.4
3,36.2
4,22.9
...,...
328,17.5
329,22.4
330,20.6
331,23.9


In [277]:
# Load the house price Dataset
boston = pd.read_csv('boston_full.csv')

# specify predictors/features
features = boston.columns[:-1].tolist()
X = pd.DataFrame(boston, columns=features)

# specify target column
Y = pd.DataFrame(boston, columns=['MEDV'])

In [278]:
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
3,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
4,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9


In [279]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
3,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
4,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43


In [280]:
Y.head()

,MEDV
0,24.0
1,21.6
2,33.4
3,36.2
4,22.9


In [281]:
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
3,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
4,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9


In [282]:
# Build Regression Model
model = RandomForestRegressor()
model.fit(X, Y)


c:\Users\Comp2\Desktop\Folder\COMPSCI_SOFTWARE_ENGINEERING\PROJECTS\project1\github\price_prediction_model\price\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [283]:
# Apply Model to Make Prediction
prediction = model.predict(boston_test)

In [284]:
# Load the house price Dataset
boston = pd.read_csv('boston_full.csv')

# specify predictors/features
features = boston.columns[:-1].tolist()
X = pd.DataFrame(boston, columns=features)

# specify target column
Y = pd.DataFrame(boston, columns=['MEDV'])

In [285]:
import pickle
import joblib

In [286]:
# Build Regression Model
model = RandomForestRegressor()
model.fit(X, Y)

with open('price_prediction_model', 'wb') as f:
    pickle.dump(model,f)


c:\Users\Comp2\Desktop\Folder\COMPSCI_SOFTWARE_ENGINEERING\PROJECTS\project1\github\price_prediction_model\price\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
